Imports, global variables and SETTINGS

In [3]:
import pandas as pd
import numpy as np
import re

#pd.set_option('display.max_rows', 500)
#pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)
#pd.set_option('display.max_colwidth', 1000)


filename = "file.csv"

columns_to_remove = "kolumnid.txt"

data = pd.DataFrame(pd.read_csv(filename, sep=";"))



########################################################SETTINGS########################################################

#Set true to use openstreetmaps to convert gps coordinates to addresses when the Address field is being corrected.
generate_missing_address_from_coords_online = False

#Set true to use openstreetmaps to convert addresses to gps coordinates when the GPS  fields are being corrected.
generate_missing_coords_from_address_online = True

#Set true to convert all coordinates from L-EST97 to WGS-84
do_convert_coordinates = True

########################################################################################################################

Removing columns we do not need:

In [ ]:
with open(columns_to_remove, encoding = "UTF-8") as file:
    for line in file:
        var = line.strip().split("|")

        if var[2].strip() == "drop":
            data = data.drop(columns=f"{var[1].strip()}")
            print(f"[{var[0].strip()}] Column {var[1].strip()} was dropped.")
        else:
            print(f"[{var[0].strip()}] Column {var[1].strip()} was not dropped.")

Fixing each column in the dataset:

Fix column "Juhtumi nr"

In [ ]:
#Replace all non-numeric characters in "Case numbers" column with zeroes.
data['Juhtumi nr'] = data['Juhtumi nr'].str.replace(pat=r'\D', repl='1', regex=True)
data.rename(columns={'Juhtumi nr' : 'Case'}, inplace=True)

#Convert to int64
data['Case'] = data['Case'].astype('int64')

#check
data['Case'].describe

Fix 'Toimumisaeg'. separate dates and times to separate columns.

In [ ]:
data["Date"], data["Time"] = data["Toimumisaeg"].str.split(" ", 1).str

data = data.drop("Toimumisaeg", axis=1)

#data['Time'] = pd.to_datetime(data['Time'], format='%H:%M').apply(pd.Timestamp).dt.time

print(data['Date'].describe)
print(data['Time'].describe)

Fix "isikuid".

In [ ]:
#Rename column
data.rename(columns={'Isikuid' : 'Involved'}, inplace=True)

#Fix null/nan values
print("Count of null values: ", data['Involved'].isna().sum())
data.loc[data['Involved'].isna(), 'Involved'] = data['Hukkunuid'] + data['Vigastatuid']
data['Involved'].fillna(0, inplace=True)
print("Count of null values: ", data['Involved'].isna().sum())

#Set type as int64
data['Involved'] = data['Involved'].astype('int64', errors='ignore')

#check results
print(data['Involved'].value_counts(normalize=False, dropna=False))


Fix "hukkunuid" and "vigastatuid".

In [ ]:
data.rename(columns={'Hukkunuid' : 'Dead', 'Vigastatuid' : 'Wounded'}, inplace=True)

print(data['Dead'].value_counts(normalize=False, dropna=False))
print(data['Wounded'].value_counts(normalize=False, dropna=False))


Fix 'Sõidukeid':

In [ ]:
#Rename column
data.rename(columns={'Sõidukeid' : 'Vehicles'}, inplace=True)

#Fill null values
data['Vehicles'].fillna(0, inplace=True)

#Fix type
data['Vehicles'] = data['Vehicles'].astype('int64', errors='ignore')

#Check
print(data['Vehicles'].value_counts(normalize=False, dropna=False))

Helper methods:

In [ ]:
# Import the necessary modules
import requests
import json
import folium
from IPython.display import display
import pyproj
import time

COUNT_CONV = 0
COUNT_GPS = 0

def incrementConversions():
    global COUNT_CONV
    COUNT_CONV = COUNT_CONV + 1

def incrementCoords():
    global COUNT_GPS
    COUNT_GPS = COUNT_GPS + 1


def get_lat_lon(address):
    # Set up the URL template
    url = "https://nominatim.openstreetmap.org/search?format=json&q={}"

    # Replace spaces in the address with "+" characters
    address = address.replace(" ", "+")

    # Format the URL with the address
    url = url.format(address)

    # Send the request and get the response data
    response = requests.get(url)
    data = response.json()

    # Check if the data list is empty
    if not data:
        # If the data list is empty, return None
        return (None, None)

    # Extract the latitude and longitude from the response data
    lat = data[0]["lat"]
    lon = data[0]["lon"]

    # Return the latitude and longitude as a tuple
    incrementCoords()
    print(COUNT_GPS, lat, lon)
    return (lat, lon)


def show_map(lat, lon):
    # Import the necessary modules

    # Create a Map object
    m = folium.Map(location=[lat, lon], zoom_start=17)

    # Display the map
    display(m)


# Set up the L-EST97 projection using the pyproj.Proj class
l_est97 = pyproj.Proj(init='epsg:3301')
# Set up the WGS-84 projection using the pyproj.Proj class
wgs84 = pyproj.Proj(init='epsg:4326')
def convert_coordinates(x, y):
    # Convert the easting and northing coordinates from L-EST97 to WGS-84 using the pyproj.transform method
    lon, lat = pyproj.transform(l_est97, wgs84, y, x)

    incrementConversions()
    print(COUNT_CONV, "Old: ", x, y, " New: ", lat, lon)
    return lat, lon


def get_address(lat, lon):
    # Set up the URL template
    url = "https://nominatim.openstreetmap.org/reverse?format=json&lat={}&lon={}"

    # Format the URL with the latitude and longitude
    url = url.format(lat, lon)

    # Send the request and get the response data
    response = requests.get(url)
    data = response.json()

    # Extract the address from the response data
    address = data.get("display_name", "")

    time.sleep(1)

    # Return the address
    return address


def create_address(est_lat, est_lon):
    coords = convert_coordinates(est_lat, est_lon)

    #Create address from coords and return it.
    return get_address(coords[0], coords[1])


def update_lat_lon(row):
    # Convert the coordinates
    t = convert_coordinates(row['Latitude'], row['Longitude'])
    
    #Return converted coordinates as series
    return pd.Series({"Latitude": t[0], "Longitude": t[1]})


def update_lat_lon_from_address(address):
    #Get cords from address
    lat, lon = get_lat_lon(address)
    #return coords as a series
    return pd.Series({"Latitude": lat, "Longitude": lon})


Example:

In [ ]:
#Test getting cords of Delta Center
#coords = get_lat_lon('Narva mnt 18 51009 Tartu')
#print(coords[0], coords[1])
#show_map(coords[0], coords[1])

#The location should be: Harju maakond Tallinn Kesklinna linnaosa Estonia pst
x = 6588678.0	
y = 542647.0
cords = convert_coordinates(x, y)
print(cords)
show_map(cords[0], cords[1])

Fix "Aadress PPA":

In [ ]:
#Rename column
data.rename(columns={'Aadress (PPA)' : 'Address'}, inplace=True)


print("Count of null values in Address column pre modification: ", data['Address'].isnull().sum())

#Create a temp column we use to quantify rows
data.insert(1, "Temp", "")

#The ones with gps but no address can use gps as we use primarily gps in our project
data['Temp'] = np.where((data["GPS X"].notnull()) & (data["GPS Y"].notnull() & (data["Address"].isnull())), "ONLY GPS", data['Temp'])

#The ones without gps but with address can have their gps constructed
data['Temp'] = np.where((data["GPS X"].isnull()) & (data["GPS Y"].isnull() & (data["Address"].notnull())), "ONLY ADDRESS", data['Temp'])

#The ones without any location data will be dropped
data['Temp'] = np.where((data["GPS X"].isnull()) & (data["GPS Y"].isnull() & (data["Address"].isnull())), "NOTHING", data['Temp'])

print("Count of null values in Address column post modification: ", data['Address'].isnull().sum())
print()
print("Count of null values with only GPS data and no address: ", len(data[data['Temp'] == "ONLY GPS"]))
print("Count of rows with only Address data and no gps: ", len(data[data['Temp'] == "ONLY ADDRESS"]))
print("Count of null values with NO location data at all: ", len(data[data['Temp'] == "NOTHING"]))

print()

#Drop the rows with no location data
data = data[data['Temp'] != "NOTHING"]

#Adds addresses to the rows that have only gps
if generate_missing_address_from_coords_online:
    data.loc[data["Temp"] == "ONLY GPS", "Address"] = data.apply(lambda row: create_address(row["GPS X"], row["GPS Y"]), axis=1)
print("Count of null values in Address column post modification: ", data['Address'].isnull().sum())

#Drop the ones we could not generate and address for
data = data[data['Address'] != ""]

#Finally, remove any remaining rows where address remained null
data = data[pd.notnull(data['Address'])]


print("Count of null values in Address column after we drop the rows we could not get addresses for: ", data['Address'].isnull().sum())


Fix "Liiklusõnnetuse liik [3]"

In [ ]:
#Rename column
data.rename(columns={'Liiklusõnnetuse liik [3]' : 'Type'}, inplace=True)

print(data['Type'].value_counts(normalize=False, dropna=False))


Fixing the parttaker fields:

In [ ]:
#Fix column names
data.rename(columns={
'Kergliikurijuhi osalusel' : 'Involved_lightmover',
'Jalakäija osalusel' : 'Involved_pedestrian',
'Kaassõitja osalusel' : 'Involved_passenger',
'Maastikusõiduki juhi osalusel' : 'Involved_offroad_vehicle',
'Eaka (65+) mootorsõidukijuhi osalusel' : 'Involved_elderly',
'Bussijuhi osalusel' : 'Involved_busdriver',
'Veoautojuhi osalusel' : 'Involved_truckdriver',
'Ühissõidukijuhi osalusel' : 'Involved_public_transport_driver',
'Sõiduautojuhi osalusel' : 'Involved_car_driver',
'Mootorratturi osalusel' : 'Involved_motorbike_driver',
'Mopeedijuhi osalusel' : 'Involved_moped_driver',
'Jalgratturi osalusel' : 'Involved_biker',
'Alaealise osalusel' : 'Involved_minor',
'Turvavarustust mitte kasutanud isiku osalusel' : 'Involved_not_wore_seatbelt',
'Esmase juhiloa omaniku osalusel' : 'Involved_learner_driver',
'Mootorsõidukijuhi osalusel' : 'Involved_vehicle_driver',
}, inplace=True)

cols = [
'Involved_lightmover',
'Involved_pedestrian',
'Involved_passenger',
'Involved_offroad_vehicle',
'Involved_elderly',
'Involved_busdriver',
'Involved_truckdriver',
'Involved_public_transport_driver',
'Involved_car_driver',
'Involved_motorbike_driver',
'Involved_moped_driver',
'Involved_biker',
'Involved_minor',
'Involved_not_wore_seatbelt',
'Involved_learner_driver',
'Involved_vehicle_driver']

#replace nans with 0-s, set all values to be if type int and remove any nan-s that might have popped up during that operation aswell
data[cols] = data[cols].replace(np.nan, 0)
data[cols] = data[cols].apply(pd.to_numeric, errors="coerce", downcast="integer")
data[cols] = data[cols].replace(np.nan, 0)

#verify results
for c in cols:
    print(data[c].value_counts(normalize=False, dropna=False))


Fix the road situation columns:

In [ ]:
#Fix column names
data.rename(columns={
'Tee tüüp [2]':'Road_type',
'Tee element [1]':'Feature',
'Kurvilisus' : 'Curvature',
'Tee tasasus':'Levelness',
'Tee seisund':'Road_condition',
'Teekatte seisund [2]':'Surface_condition',
'Lubatud sõidukiirus (PPA)':'Speed_limit'
}, inplace=True)


cols = [
'Road_type',
'Feature',
'Curvature',
'Levelness',
'Road_condition',
'Surface_condition',
'Speed_limit'
]

#Set 0 speed limits as NaN as Estonia has no motorways without speed limits
data['Speed_limit'] = data['Speed_limit'].replace(0, np.nan)

#Fix a probable typo
data['Speed_limit'][data['Speed_limit'] == 901] = 90


#check results
print(data.shape)
print()
for c in cols:
    print(data[c].value_counts(normalize=False, dropna=False))
    print()

Fix weather and lighting:

In [ ]:
#Fix column names
data.rename(columns={
'Ilmastik [1]':'Weather',
'Valgustus [2]':'Lighting'
}, inplace=True)


cols = ['Weather', 'Lighting']

#data = data.dropna(subset=cols)

#check results
print(data.shape)
print()
for c in cols:
    print(data[c].value_counts(normalize=False, dropna=False))
    print()

Fix gps:

In [ ]:
#Fix column names
data.rename(columns={
'GPS X':'Latitude',
'GPS Y':'Longitude'
}, inplace=True)

print("Latitude nan count: ", data['Latitude'].isna().sum(), "Longitude nan count: ", data['Longitude'].isna().sum())

#first we must convert all we can, then create coords for the rows that have address, afterwards drop rest what we cant generate for

#Convert all coordinates from L-EST97 to WGS-84
if do_convert_coordinates:
    print("Now vonverting co-ordinates >>> ")
    data[["Latitude", "Longitude"]] = data.apply(update_lat_lon, axis=1)
    
    

#Try to generate missing coordinates from address data if permitted
if generate_missing_coords_from_address_online:
    print("Now generating co-ordinats from Address-es >>> ")
    #data[["Latitude", "Longitude"]] = data.apply(lambda row: update_lat_lon_from_address(row["Address"]), axis=1)
    # Select the rows where the "Temp" column has the value "ONLY ADDRESS", so we know which rows to try to create coords for
    rows = data.loc[data["Temp"] == "ONLY ADDRESS"]
    # Apply the update_lat_lon_from_address() function to the selected rows
    rows[["Latitude", "Longitude"]] = rows.apply(lambda row: update_lat_lon_from_address(row["Address"]), axis=1)
    data.update(rows)


#Finally remove any nan-s from rows where we did not generate coords from address or where generating was impossible
cols = ['Latitude', 'Longitude']
data = data.dropna(subset=cols)

Remove temp column and drop any last nan rows, plus output the dataframe to a new csv file:

In [ ]:
data.drop(['Temp'], axis=1, inplace=True)
#data.dropna(inplace=True)

data.to_csv('cleaned_data_file.csv', encoding='utf-8', index=False, header=True)

Final result:

In [ ]:
print('Shape', data.shape)
print("Latitude nan count: ", data['Latitude'].isna().sum(), "Longitude nan count: ", data['Longitude'].isna().sum())
data.head()